In [9]:
def search_sequence(request):
    query = request.GET.get('query_sequence')

    model = cache.get('model')
    batch_converter = cache.get('batch_converter')

    dataset = cache.get('dataset')
    index = cache.get('faiss_index')

    if dataset is None:
        dataset = load_from_disk('9606') #server is run from the upper directory
        columns_to_include = ['id', 'annotation', 'alias', 'seq_len', 't_sne']
        data = [{col: row[col] for col in columns_to_include} for row in dataset]
        cache.set('dataset', data)

    if model is None or batch_converter is None:
        model, alphabet = pretrained.esm2_t30_150M_UR50D()
        batch_converter = alphabet.get_batch_converter()
        device = torch.device("cuda:0")
        model.to(device)
        cache.set('model', model)
        cache.set('batch_converter', batch_converter)

    if index ==None:
        index = faiss.IndexHNSWFlat(640, 512, faiss.METRIC_L2)
        dataset.add_faiss_index("embeddings", custom_index=index)
        index.set('index', index)

    idx = retrieve_similar_sequences(query, dataset, return_num=200, 
                                     batch_converter=batch_converter, model=model)

    result = indices_to_info(idx, dataset, annotation_len=400)

    return JsonResponse(result, safe=False)

In [7]:
from django.http import JsonResponse
from datasets import load_dataset, load_from_disk
from django.core.cache import cache
from django.views.decorators.http import require_http_methods
from esm import pretrained
import torch
import faiss
